In [1]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch.autograd import Variable
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn as nn
import urllib.request
import numpy as np
import random
import struct
import torch
import errno
import math
import gzip
import io
import os

In [2]:
class fmnist(Dataset):
#Referred from https://github.com/zalandoresearch/fashion-mnist
#Referred from https://github.com/ashmeet13/FashionMNIST-CNN/tree/master
    urls = ['http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz',
    'http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz', 'http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-images-idx3-ubyte.gz',
    'http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz']

    file_name =['train-images-idx3-ubyte', 'train-labels-idx1-ubyte', 't10k-images-idx3-ubyte', 't10k-labels-idx1-ubyte']

    raw="raw"
    processed="processed"


    def __init__(self, root, train=True, transform=True, download=True):
        super(fmnist, self).__init__()
        self.root = root
        self.train=train
        self.transform=transform
        #transform and standardize the image to tensor
        self.tensor_transform = transforms.ToTensor()
        raw_path = os.path.join(self.root,self.raw)

        if download and (os.path.exists(raw_path) == False):
            self.download(self.root)

        if self.train:
            train_path = os.path.join(self.root,self.processed,"training_set.pt")
            self.train_images, self.train_labels = torch.load(train_path)
        else:
            test_path = os.path.join(self.root,self.processed,"testing_set.pt")
            self.test_images, self.test_labels = torch.load(test_path)

    def __getitem__(self,index):
        if self.train:
            image, label = self.train_images[index], self.train_labels[index]
        else:
            image, label = self.test_images[index], self.test_labels[index]

        image = image.numpy()
        image = np.rot90(image,axes = (1,2)).copy()

        if self.transform and self.train:
            image = self.transform_process(image)

        image = self.tensor_transform(image)
        image = image.contiguous()
        image = image.view(1,28,28)
        return image,label


    def __len__(self):
        if self.train:
            return(len(self.train_images))
        else:
            return(len(self.test_images))


    def transform_process(self, image): #Would or would not return a flipped image
        self.rotate = random.getrandbits(1)
        image = np.flip(image,self.rotate).copy()
        return image

    '''
    download(root) -> The function will download and save the MNIST images in raw
    format under the 'raw' folder under the user specified root directory
    '''

    def download(self, root):
        raw_path = os.path.join(self.root,self.raw)
        processed_path = os.path.join(self.root,self.processed)

        try:
            os.makedirs(raw_path)
            os.makedirs(processed_path)
        except OSError as exc:
            if exc.errno != errno.EEXIST:
                raise
            pass

        for file_index in range(len(self.file_name)):
            print("Downloading:",self.urls[file_index])
            urllib.request.urlretrieve(self.urls[file_index],(self.file_name[file_index]+'.gz'))
            print("Extracting:",self.file_name[file_index]+".gz")
            f = gzip.open(self.file_name[file_index]+'.gz', 'rb')
            with open(raw_path+"/"+self.file_name[file_index],'wb') as w:
                for line in f.readlines():
                    w.write(line)
            f.close()
            os.remove(self.file_name[file_index]+".gz")

        print()
        print("Raw data downloaded and extracted in your specified root directory under /raw")
        print()
        self.process(self.root)

    '''
    process(root) -> Will process the raw downloaded files into a usable format
    and store them into the a 'processed' folder under user specified root
    directory.
    '''

    def process(self, root):
        raw_path = os.path.join(self.root,self.raw)
        processed_path = os.path.join(self.root,self.processed)

        print("Processing training data")
        train_image = self.readimg(self.root, self.file_name[0], 2051)
        train_label	= self.readlab(self.root, self.file_name[1], 2049)
        train_data = (train_image,train_label)

        print("Processing testing data")
        test_image = self.readimg(self.root, self.file_name[2], 2051)
        test_label = self.readlab(self.root, self.file_name[3], 2049)
        test_data = (test_image,test_label)

        train_path = os.path.join(self.root,self.processed,"training_set.pt")
        with open(train_path,"wb") as f:
            torch.save(train_data,f)

        test_path = os.path.join(self.root,self.processed,"testing_set.pt")
        with open(test_path,"wb") as f:
            torch.save(test_data,f)
        print()
        print("Processed data has been stored in your specified root directory under /processsed")
        print()


    def readimg(self, root, file, magic):
        image = []
        path = os.path.join(self.root,self.raw,file)
        with open(path,'rb') as f:
            magic_number, size, row, col = struct.unpack('>IIII',f.read(16))
            assert (magic_number == magic)
            for run in range(size*row*col):
                image.append(list(struct.unpack('B',f.read(1)))[0])
            image = np.asarray(image, dtype = np.float32)
            return (torch.from_numpy(image).view(size,1,row,col))


    def readlab(self, root, file, magic):
        label = []
        path = os.path.join(self.root,self.raw,file)
        with open(path,'rb') as f:
            magic_number, size = struct.unpack(">II",f.read(8))
            assert (magic_number == magic)
            for run in range(size):
                label.append(list(struct.unpack('b',f.read(1)))[0])
            label = np.asarray(label)
            return (torch.from_numpy(label))


In [3]:
#Extract the training and testing dataset in a folder named Fashion MNIS
train_dataset = fmnist(root = ".\FashionMNIS", train = True, transform = True, download = True)
test_dataset = fmnist(root = ".\FashionMNIS", train = False, transform = False, download = True)

Downloading: http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz
Extracting: train-images-idx3-ubyte.gz
Downloading: http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz
Extracting: train-labels-idx1-ubyte.gz
Downloading: http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-images-idx3-ubyte.gz
Extracting: t10k-images-idx3-ubyte.gz
Downloading: http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz
Extracting: t10k-labels-idx1-ubyte.gz

Raw data downloaded and extracted in your specified root directory under /raw

Processing training data
Processing testing data

Processed data has been stored in your specified root directory under /processsed



In [4]:
#Calculation of total epochs using a defined batch size(batch_size)
#and total iterations(n_ters)


batch_size = 100
n_iters = 18000
epoch_size = n_iters/(len(train_dataset)/batch_size)

In [5]:
#Convolutional Networks
class CNNModel(nn.Module):
    def __init__(self):
        super (CNNModel, self).__init__()

#Convolution 1  with kernel size 3
        self.cnn1 = nn.Conv2d(in_channels = 1, out_channels = 32, kernel_size = 3, stride = 1, padding = 2)
        self.relu1 = nn.ReLU()

#Batch Normalization
        self.norm1 = nn.BatchNorm2d(32)
        nn.init.xavier_uniform(self.cnn1.weight)

#Max pooling for reducing dimensions
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)

#Second convolution to reduce and sharpen the image further
        self.cnn2 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, stride = 1, padding = 2)
        self.relu2 = nn.ReLU()

#Batch Normalization
        self.norm2 = nn.BatchNorm2d(64)
        nn.init.xavier_uniform(self.cnn2.weight)

#Max Pooling again for reduction of pixels
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)


        self.fc1 = nn.Linear(4096, 4096)
        self.fcrelu = nn.ReLU()

        self.fc2 = nn.Linear(4096, 10)

    def forward(self, x):
        out = self.cnn1(x)
        out = self.relu1(out)
        out = self.norm1(out)

        out = self.maxpool1(out)

        out = self.cnn2(out)
        out = self.relu2(out)
        out = self.norm2(out)

        out = self.maxpool2(out)

        out = out.view(out.size(0),-1)

        out = self.fc1(out)
        out = self.fcrelu(out)

        out = self.fc2(out)
        return out

In [6]:
#Load the model
model = CNNModel()
if torch.cuda.is_available():
    model.cuda()

<ipython-input-5-aa0e945d6b5e>:8: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(self.cnn1.weight)
<ipython-input-5-aa0e945d6b5e>:15: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(self.cnn2.weight)


In [7]:
#I have used Cross Entropy for Categorical Predictions
criterion = nn.CrossEntropyLoss()

learning_rate = 0.015
moment = 0.9
optimizer = optim.SGD(model.parameters(),lr = learning_rate, momentum = moment, nesterov = True)

In [8]:

iter = 0
for epoch in range(int(math.ceil(epoch_size))):

    '''
    Loading the training dataset after every epoch which will
    load it from the Fashion Dataset Class making a new train
    loader for every new epoch causing random flips and random
    shuffling of above exampled Fashion MNIST images.
    '''

    train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)
    test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = True)
    for i, (images, labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
        else:
            images = Variable(images)
            labels = Variable(labels)
        optimizer.zero_grad()
        labels=labels.long()
        outputs = model(images)

        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        iter += 1

#At every 3000th epoch a test on the above initialised test dataset
#(test_loader) would be performed and an accuracy would be provided.


        if iter%3000 == 0:
            correct = 0
            total = 0
            for image,label in test_loader:
                if torch.cuda.is_available():
                    image = Variable(image.cuda())
                else:
                    image = Variable(image)
                output = model(image)
                _, predicted = torch.max(output.data,1)
                total += label.size(0)
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == label.cpu()).sum()
                else:
                    correct += (predicted == label).sum()

            accuracy = 100 * (correct/total)
            print('Iteration: {} Loss: {} Accuracy: {}'.format(iter, loss.data, accuracy))

Iteration: 3000 Loss: 0.19170606136322021 Accuracy: 92.29999542236328
Iteration: 6000 Loss: 0.07875138521194458 Accuracy: 92.33999633789062
Iteration: 9000 Loss: 0.03824256360530853 Accuracy: 92.51000213623047
Iteration: 12000 Loss: 0.002043476328253746 Accuracy: 93.0199966430664
Iteration: 15000 Loss: 0.00025225349236279726 Accuracy: 93.41999816894531
Iteration: 18000 Loss: 5.1569280913099647e-05 Accuracy: 93.51000213623047


In [9]:
#Saving Model at the current trained state
PATH= "model1.pt"
torch.save({
            'epoch': epoch_size,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': criterion
            }, PATH)

In [17]:
#SCRIPT for opening the model again

PATH= "model1.pt"
model1 = CNNModel()
optimizer1 = optim.SGD(model.parameters(),lr = 0.015, momentum = 0.9, nesterov = True)
checkpoint = torch.load(PATH)

#Saving Model Weights
torch.save(model1.load_state_dict(checkpoint['model_state_dict']),'model1_weights.pth')
optimizer1.load_state_dict(checkpoint['optimizer_state_dict'])
epoch1 = checkpoint['epoch']
loss1 = checkpoint['loss']
model1.eval()
model1.cnn1.weight
model1.cnn2.weight

<ipython-input-5-aa0e945d6b5e>:8: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(self.cnn1.weight)
<ipython-input-5-aa0e945d6b5e>:15: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(self.cnn2.weight)


Parameter containing:
tensor([[[[ 0.0183,  0.0223, -0.0603],
          [-0.0250, -0.0659, -0.0413],
          [-0.0332, -0.0053, -0.0545]],

         [[-0.0327,  0.0412,  0.0217],
          [ 0.0748, -0.0031,  0.0418],
          [ 0.0595, -0.0034, -0.1220]],

         [[-0.0628,  0.0176, -0.0930],
          [-0.0778, -0.0346,  0.0551],
          [-0.1073, -0.0338,  0.0139]],

         ...,

         [[-0.0463,  0.0374,  0.0067],
          [ 0.0590,  0.0569,  0.0302],
          [-0.0045,  0.0022,  0.0411]],

         [[-0.1131, -0.0489, -0.0631],
          [ 0.0268, -0.0382, -0.0685],
          [ 0.0251, -0.1256,  0.0401]],

         [[ 0.0395,  0.1339, -0.0135],
          [ 0.0434,  0.1039,  0.0779],
          [ 0.0393, -0.0955, -0.0250]]],


        [[[-0.0970,  0.0368,  0.0346],
          [ 0.0750,  0.0374, -0.0094],
          [-0.0085,  0.0495, -0.0008]],

         [[ 0.0033, -0.0360, -0.0971],
          [-0.0382,  0.0776,  0.0135],
          [ 0.0976, -0.0691,  0.0532]],

         